In [268]:
#Learning MLPipelines using spark

In [269]:
##Example for writing functions that handles errors. We use this function to
##convert each element of the row to double.

In [270]:
rawData = (sc.textFile(path)
           .map(lambda x : x.split(","))
           .map(lambda l : map(lambda element: typeConvert(element),l))
           .take(1)
            )
print rawData

[[39.0, u'State-gov', 77516.0, u'Bachelors', 13.0, u'Never-married', u'Adm-clerical', u'Not-in-family', u'White', u'Male', 2174.0, 0.0, 40.0, u'United-States', u'<=50K']]


In [271]:
def typeConvert(value):
    try:
        return float(value)
    except ValueError,TypeError:
        return (value.strip())


In [272]:
path = '/Users/vishnu/Documents/datasets/myexperiments/adult/adult.data'
rawData = (sc.textFile(path)
           .map(lambda x : x.split(","))
           .map(lambda l : map(lambda element: typeConvert(element),l))
            )
    
rawData.first()

[39.0,
 u'State-gov',
 77516.0,
 u'Bachelors',
 13.0,
 u'Never-married',
 u'Adm-clerical',
 u'Not-in-family',
 u'White',
 u'Male',
 2174.0,
 0.0,
 40.0,
 u'United-States',
 u'<=50K']

In [273]:
newRawData = (sc.textFile(path)
           .map(lambda x : x.split(","))
           .map(lambda x : len(x))
           .distinct()
           .collect()
             )

In [274]:
print newRawData

[1, 15]


In [275]:
newRawData = (sc.textFile(path)
           .map(lambda x : x.split(","))
           .filter(lambda x : len(x)==1)
            .take(10) )
print newRawData

[[u'']]


In [276]:
rawData = (sc.textFile(path)
           .map(lambda x : x.split(","))
           .filter(lambda x : len(x)==15)
           .map(lambda l : map(lambda element: typeConvert(element),l))
            )

In [277]:
rawData.first()

[39.0,
 u'State-gov',
 77516.0,
 u'Bachelors',
 13.0,
 u'Never-married',
 u'Adm-clerical',
 u'Not-in-family',
 u'White',
 u'Male',
 2174.0,
 0.0,
 40.0,
 u'United-States',
 u'<=50K']

In [278]:
##Creating Dataframe

In [279]:
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType ,DoubleType ,FloatType

In [363]:
schema = StructType([
    StructField("age",FloatType(),True),
    StructField("workclass",StringType(),True),
    StructField("fnlwgt",DoubleType(),True),
    StructField("education",StringType(),True),
    StructField("unknown",StringType(),True),  
    StructField("marital_status",StringType(),True),
    StructField("occupation",StringType(),True),
    StructField("relationship",StringType(),True),
    StructField("race",StringType(),True),
    StructField("sex",StringType(),True),
    StructField("capital_gain",DoubleType(),True),
    StructField("capital_loss",DoubleType(),True),
    StructField("hours_per_week",DoubleType(),True),
    StructField("native_country",StringType(),True),
    StructField("income",StringType(),True)
])

In [364]:
 # Create a DataFrame by applying the schema to the RDD and print the schema

In [365]:
rawdf = sqlContext.createDataFrame(rawData, schema)

In [366]:
rawdf.printSchema()

root
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- unknown: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)



In [395]:
rawdf = rawdf.drop('unknown')

In [396]:
#rawdf.groupBy("workclass").count().show()
rawdf.groupBy("workclass").count().show()


+----------------+-----+
|       workclass|count|
+----------------+-----+
|         Private|24532|
|     Federal-gov|  960|
|       State-gov| 1298|
|       Local-gov| 2093|
|    Self-emp-inc| 1116|
|     Without-pay|   14|
|    Never-worked|    7|
|Self-emp-not-inc| 2541|
+----------------+-----+



In [397]:
from pyspark.sql.functions import *
rawdf.groupBy("native_country").count().orderBy("count", ascending=False).show()

+------------------+-----+
|    native_country|count|
+------------------+-----+
|     United-States|29170|
|            Mexico|  643|
|           Private|  583|
|       Philippines|  198|
|           Germany|  137|
|            Canada|  121|
|       Puerto-Rico|  114|
|       El-Salvador|  106|
|             India|  100|
|              Cuba|   95|
|           England|   90|
|           Jamaica|   81|
|             South|   80|
|             China|   75|
|             Italy|   73|
|Dominican-Republic|   70|
|           Vietnam|   67|
|         Guatemala|   64|
|             Japan|   62|
|            Poland|   60|
+------------------+-----+
only showing top 20 rows



In [398]:
rawdf.groupBy("occupation").count().show(truncate=False)

+-----------------+-----+
|occupation       |count|
+-----------------+-----+
|Other-service    |3295 |
|Prof-specialty   |4140 |
|Farming-fishing  |994  |
|?                |1843 |
|Protective-serv  |649  |
|Adm-clerical     |3770 |
|Machine-op-inspct|2002 |
|Transport-moving |1597 |
|Sales            |3650 |
|Priv-house-serv  |149  |
|Handlers-cleaners|1370 |
|Craft-repair     |4099 |
|Exec-managerial  |4066 |
|Tech-support     |928  |
|Armed-Forces     |9    |
+-----------------+-----+



In [399]:
rawdf = rawdf.na.replace(['?'],['Private'],'workclass')
rawdf = rawdf.na.replace(['?'],['Private'],'native_country')


In [400]:
rawdf.groupBy("workclass").count().show()

+----------------+-----+
|       workclass|count|
+----------------+-----+
|         Private|24532|
|     Federal-gov|  960|
|       State-gov| 1298|
|       Local-gov| 2093|
|    Self-emp-inc| 1116|
|     Without-pay|   14|
|    Never-worked|    7|
|Self-emp-not-inc| 2541|
+----------------+-----+



In [401]:
rawdf.groupBy("native_country").count().orderBy("count",ascending=False).show()

+------------------+-----+
|    native_country|count|
+------------------+-----+
|     United-States|29170|
|            Mexico|  643|
|           Private|  583|
|       Philippines|  198|
|           Germany|  137|
|            Canada|  121|
|       Puerto-Rico|  114|
|       El-Salvador|  106|
|             India|  100|
|              Cuba|   95|
|           England|   90|
|           Jamaica|   81|
|             South|   80|
|             China|   75|
|             Italy|   73|
|Dominican-Republic|   70|
|           Vietnam|   67|
|         Guatemala|   64|
|             Japan|   62|
|            Poland|   60|
+------------------+-----+
only showing top 20 rows



In [402]:
from pyspark.ml.feature import *

In [403]:
from pyspark.ml.feature import *
def indexStringColumns(df,cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col,outputCol=col+"-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf).drop(col)
        tempdf = tempdf.withColumnRenamed(col+"-num",col)
    return tempdf

In [404]:
cols = {"workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "income"}
dfnumeric = indexStringColumns(rawdf,cols)
print cols
dummy.printSchema()

set(['native_country', 'relationship', 'marital_status', 'sex', 'race', 'income', 'workclass', 'education', 'occupation'])
root
 |-- age: float (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: double (nullable = true)
 |-- relationship: double (nullable = true)
 |-- marital_status: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- race: double (nullable = true)
 |-- income: double (nullable = true)
 |-- workclass: double (nullable = true)
 |-- education: double (nullable = true)
 |-- occupation: double (nullable = true)



In [405]:
(rawdf.select("workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "income")
      .show())

+----------------+------------+--------------------+-----------------+-------------+------------------+------+--------------+------+
|       workclass|   education|      marital_status|       occupation| relationship|              race|   sex|native_country|income|
+----------------+------------+--------------------+-----------------+-------------+------------------+------+--------------+------+
|       State-gov|   Bachelors|       Never-married|     Adm-clerical|Not-in-family|             White|  Male| United-States| <=50K|
|Self-emp-not-inc|   Bachelors|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male| United-States| <=50K|
|         Private|     HS-grad|            Divorced|Handlers-cleaners|Not-in-family|             White|  Male| United-States| <=50K|
|         Private|        11th|  Married-civ-spouse|Handlers-cleaners|      Husband|             Black|  Male| United-States| <=50K|
|         Private|   Bachelors|  Married-civ-spouse|   Prof-specialty

In [406]:
(dfnumeric.select("workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "income")
      .show())

+---------+---------+--------------+----------+------------+----+---+--------------+------+
|workclass|education|marital_status|occupation|relationship|race|sex|native_country|income|
+---------+---------+--------------+----------+------------+----+---+--------------+------+
|      3.0|      2.0|           1.0|       3.0|         1.0| 0.0|0.0|           0.0|   0.0|
|      1.0|      2.0|           0.0|       2.0|         0.0| 0.0|0.0|           0.0|   0.0|
|      0.0|      0.0|           2.0|       9.0|         1.0| 0.0|0.0|           0.0|   0.0|
|      0.0|      5.0|           0.0|       9.0|         0.0| 1.0|0.0|           0.0|   0.0|
|      0.0|      2.0|           0.0|       0.0|         4.0| 1.0|1.0|           9.0|   0.0|
|      0.0|      3.0|           0.0|       2.0|         4.0| 0.0|1.0|           0.0|   0.0|
|      0.0|     10.0|           5.0|       5.0|         1.0| 1.0|1.0|          11.0|   0.0|
|      1.0|      0.0|           0.0|       2.0|         0.0| 0.0|0.0|           

In [407]:
def oneHotEncodeColumns(df,cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(inputCol=col,outputCol=col+"-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col)
        tempdf = tempdf.withColumnRenamed(col+"-onehot",col)
    return tempdf

In [408]:
dfhot = oneHotEncodeColumns(dfnumeric,{"workclass", "education", "marital_status", "occupation", "relationship", "race", "native_country"})
dfhot.printSchema()

root
 |-- age: float (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- income: double (nullable = true)
 |-- native_country: vector (nullable = true)
 |-- relationship: vector (nullable = true)
 |-- marital_status: vector (nullable = true)
 |-- race: vector (nullable = true)
 |-- workclass: vector (nullable = true)
 |-- education: vector (nullable = true)
 |-- occupation: vector (nullable = true)



In [409]:
dfhot.select("workclass", "education", "marital_status", "occupation", "relationship", "race", "native_country").show()

+-------------+---------------+--------------+---------------+-------------+-------------+---------------+
|    workclass|      education|marital_status|     occupation| relationship|         race| native_country|
+-------------+---------------+--------------+---------------+-------------+-------------+---------------+
|(7,[3],[1.0])| (15,[2],[1.0])| (6,[1],[1.0])| (14,[3],[1.0])|(5,[1],[1.0])|(4,[0],[1.0])| (41,[0],[1.0])|
|(7,[1],[1.0])| (15,[2],[1.0])| (6,[0],[1.0])| (14,[2],[1.0])|(5,[0],[1.0])|(4,[0],[1.0])| (41,[0],[1.0])|
|(7,[0],[1.0])| (15,[0],[1.0])| (6,[2],[1.0])| (14,[9],[1.0])|(5,[1],[1.0])|(4,[0],[1.0])| (41,[0],[1.0])|
|(7,[0],[1.0])| (15,[5],[1.0])| (6,[0],[1.0])| (14,[9],[1.0])|(5,[0],[1.0])|(4,[1],[1.0])| (41,[0],[1.0])|
|(7,[0],[1.0])| (15,[2],[1.0])| (6,[0],[1.0])| (14,[0],[1.0])|(5,[4],[1.0])|(4,[1],[1.0])| (41,[9],[1.0])|
|(7,[0],[1.0])| (15,[3],[1.0])| (6,[0],[1.0])| (14,[2],[1.0])|(5,[4],[1.0])|(4,[0],[1.0])| (41,[0],[1.0])|
|(7,[0],[1.0])|(15,[10],[1.0])| (6,[5

In [410]:
dfhot.printSchema()


root
 |-- age: float (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- sex: double (nullable = true)
 |-- income: double (nullable = true)
 |-- native_country: vector (nullable = true)
 |-- relationship: vector (nullable = true)
 |-- marital_status: vector (nullable = true)
 |-- race: vector (nullable = true)
 |-- workclass: vector (nullable = true)
 |-- education: vector (nullable = true)
 |-- occupation: vector (nullable = true)



In [412]:
colList = dfhot.columns
colList.remove('income')

In [414]:
vecAssembler = VectorAssembler(inputCols=colList,outputCol="features")
lpoints = vecAssembler.transform(dfhot).select("features", "income").withColumnRenamed("income", "label")

In [384]:
dfhot.select('capital_gain').groupBy('capital_gain').count().show()

+------------+-----+
|capital_gain|count|
+------------+-----+
|     15024.0|  347|
|      2202.0|   16|
|      2977.0|    8|
|     41310.0|    2|
|      4508.0|   12|
|      4101.0|   20|
|      3411.0|   24|
|     25236.0|   11|
|      2961.0|    3|
|      2936.0|    3|
|      1409.0|    7|
|      2009.0|    3|
|      2329.0|    6|
|      4787.0|   23|
|      2829.0|   31|
|      2354.0|   11|
|      1151.0|    8|
|      4687.0|    3|
|      2463.0|   11|
|      5455.0|   11|
+------------+-----+
only showing top 20 rows

